In [5]:
# import os
# os.chdir('..')
from src.modules import connect_lhl, test_models

In [6]:
conn = connect_lhl.lhl_delay_data('lhl-data-bootcamp.crzjul5qln0e.ca-central-1.rds.amazonaws.com','mid_term_project','lhl_student','lhl_student', 0)
conn.connection()
df = conn.sample_table('flights_test', samples='100000')

DatabaseError: Execution failed on sql 'SELECT SETSEED(0); SELECT * FROM flights_test WHERE flights_test.unique_carrier IN (SELECT op_unique_carrier FROM flights) ORDER BY RANDOM() LIMIT 100000;': column flights_test.unique_carrier does not exist
LINE 1: ...LECT SETSEED(0); SELECT * FROM flights_test WHERE flights_te...
                                                             ^
HINT:  Perhaps you meant to reference the column "flights_test.op_unique_carrier".
